In [1]:
import sqlite3
import random

conn = sqlite3.connect("test.db")

# function for select all row factory


def select_all_rf(table: str) -> list:
    """Returns a list of row factory objects"""

    conn.row_factory = sqlite3.Row
    c = conn.cursor()

    c.execute(f"select rowid, * from {table}")
    rows = c.fetchall()  # also works with fetchone
    conn.commit()

    return rows


def print_select_all(table: str) -> None:
    """convenience function that wraps ```select_all_rf``` and
    prints returned rows as dict"""

    for row in select_all_rf(table):
        print(dict(row))



ENABLE WAL MODE

In [4]:
c = conn.cursor()
c.execute("pragma journal_mode=WAL")
conn.commit()


### TABLES

CREATE TABLE

In [100]:
c = conn.cursor()
c.execute(
    """create table if not exists entries (entry_id integer primary key, driver_id integer, miles, kms,
    foreign key (driver_id) references drivers (driver_id))"""
)
c.execute("""create table if not exists drivers (driver_id integer primary key, driver_name)
    """
)  # if not exists is needed to stop operational error if exists
conn.commit()



DROP TABLE

In [96]:
c = conn.cursor()

c.execute("drop table if exists entries")
c.execute("drop table if exists drivers")
conn.commit()

In [102]:
c = conn.cursor()
c.execute("select * from entries limit 1")
sch = c.fetchall()
conn.commit()

sch[0].keys()

IndexError: list index out of range

### CREATE

INSERT

In [103]:
# add a test row, does sqlite create rowids, yes

c = conn.cursor()
driver = "Ben"
miles = random.randint(100, 1000)
kms = miles / 0.621371
c.execute(f"insert into entries (driver_id, miles, kms) values ('{driver}', {miles}, {kms})") # ORDER MATTERS

conn.commit()

print_select_all("entries")


{'entry_id': 1, 'driver_id': 'Ben', 'miles': 939, 'kms': 1511.174483521117}


In [106]:
c = conn.cursor()
c.execute("insert into drivers (driver_name) values ('Benjay')")
conn.commit()

### UPDATE

UPDATE

In [21]:
c = conn.cursor()
c.execute("update entries set kms = 22222 where rowid=2")
conn.commit()

print_select_all("entries")


{'rowid': 1, 'miles': 210, 'kms': 337.9623445574383}
{'rowid': 2, 'miles': 271, 'kms': 22222}
{'rowid': 3, 'miles': 772, 'kms': 1242.413952373059}


### DELETE

DELETE

In [22]:
c = conn.cursor()
c.execute("delete from entries where rowid=3")
conn.commit()

print_select_all("entries")


{'rowid': 1, 'miles': 210, 'kms': 337.9623445574383}
{'rowid': 2, 'miles': 271, 'kms': 22222}


### READ

ROWFACTORY FETCH MANY

In [7]:
conn.row_factory = sqlite3.Row
c = conn.cursor()

c.execute("select rowid, * from entries")
rows = c.fetchall()  # also works with fetchone
conn.commit()
for row in rows:
    print(type(row))
    print(dict(row))


<class 'sqlite3.Row'>
{'rowid': 1, 'miles': 210, 'kms': 337.9623445574383}
